<a href="https://colab.research.google.com/github/sharmatec/Hello-World/blob/master/ASSIGNMENT2_1098969.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd


from sklearn.model_selection import train_test_split


import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from keras.utils import to_categorical

import nltk
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Using TensorFlow backend.


Import Movie reviews dataset

In [0]:

data = pd.read_csv('https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv', sep='\t')

# Pre processing the text to remove stop words and perform lematization and cleaning the words.

In [3]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 

pd.set_option('max_colwidth',400)

wordnet = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations = "?:!.,;-()"

raw_reviews = data.Phrase.values
cleaned_reviews = []

for i in range(len(raw_reviews)):
  review = str(raw_reviews[i])
  review=re.sub('[^a-zA-Z]',' ',review)
  review=[wordnet.lemmatize(w) for w in word_tokenize(str(review).lower())]
  review=' '.join(review)
  cleaned_reviews.append(review)

data['cleaned_reviews'] = cleaned_reviews

data.head()

,PhraseId,SentenceId,Phrase,Sentiment,cleaned_reviews
0,1,1,"A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .",1,a series of escapade demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amount to much of a story
1,2,1,A series of escapades demonstrating the adage that what is good for the goose,2,a series of escapade demonstrating the adage that what is good for the goose
2,3,1,A series,2,a series
3,4,1,A,2,a
4,5,1,series,2,series


# Cleaning and catogorizing the dataset,import and intilaize vectorization

In [0]:
X = data.cleaned_reviews.values
Y = to_categorical(data.Sentiment.values)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = TfidfVectorizer(max_features=4000,stop_words = None, ngram_range=(1,2))
X = vectorizer.fit_transform(X)

Spliting dataset

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=2003)

Convert the training and test dataset into array

In [0]:
import numpy as np

x_train_np = x_train.toarray()
y_train_np = np.array(y_train)

x_test_np = x_test.toarray()
y_test_np = np.array(y_test)

Change testing and training axis

In [0]:
x_train = np.expand_dims(x_train_np, axis=2)
x_test = np.expand_dims(x_test_np, axis=2)

Performing convolution and its layer(Network connection).

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

model = Sequential()

def cnn(data_x):
  #Convolution layer 1
  model.add(Conv1D(filters = 64, kernel_size=3, activation='relu', input_shape=(data_x.shape[1],1)))
  #MaxPool layer
  model.add(MaxPooling1D(pool_size =2))
  #Convolution layer 2
  model.add(Conv1D(filters = 128, kernel_size=3, activation='relu'))
  #MaxPool layer
  model.add(MaxPooling1D(pool_size =2))
  model.add(Flatten())
  #Fully connected layer
  model.add(Dense(100, activation='relu'))
  #Dropout layer
  model.add(Dropout(0.2))
  #Output Layer
  model.add(Dense(5, activation='softmax')) 






Import metrics for Evaluation.

In [0]:
from keras import metrics
from sklearn.metrics import f1_score

Define accuracy, precision, recall and f1 Score

In [0]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Intialize categorical accuracy.

In [11]:

cnn(x_train_np)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[metrics.categorical_accuracy,f1_m,precision_m, recall_m])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Set Epochs and batch size 

In [12]:
history = model.fit(x_train, y_train_np, epochs=10, batch_size=128)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/5





109242/109242 [==============================] - 40s 367us/step - loss: 1.0684 - categorical_accuracy: 0.5746 - f1_m: 0.4894 - precision_m: 0.6718 - recall_m: 0.3911
Epoch 2/5
109242/109242 [==============================] - 27s 245us/step - loss: 0.9362 - categorical_accuracy: 0.6263 - f1_m: 0.5772 - precision_m: 0.6983 - recall_m: 0.4927
Epoch 3/5
109242/109242 [==============================] - 27s 244us/step - loss: 0.8716 - categorical_accuracy: 0.6545 - f1_m: 0.6190 - precision_m: 0.7127 - recall_m: 0.5478
Epoch 4/5
109242/109242 [==============================] - 27s 243us/step - loss: 0.8214 - categorical_accuracy: 0.6742 - f1_m: 0.6470 - precision_m: 0.7228 - recall_m: 0.5863
Epoch 5/5
109242/109242 [==============================] - 27s 244us/step - loss: 0.7809 - categorical_accuracy: 0.6902 - f1_m: 0.6698 - precision_m: 0.7338 - recall_m: 0.6166


Print Accuracy, precision, recall, f1 Score.

In [13]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy " + str(accuracy) + ":\n\ff1_score = " + str(f1_score) +
          "\nPrecision = " + str(precision) + "\nRecall = " + str(recall))

Accuracy 0.6425947285232175:
f1_score = 0.6233285477549555
Precision = 0.6732174829842111
Recall = 0.5815711905677303


General training model and print.

In [15]:
model.save("1098969_1dconv_reg")
print("Save in Drive")

Save in Drive


Printing the train model view 

In [17]:

from numpy import loadtxt
from keras.models import load_model

load_model('1098969_1dconv_reg', custom_objects={'f1_m': f1_m, 'precision_m':precision_m, 'recall_m':recall_m})

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 3998, 64)          256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1999, 64)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1997, 128)         24704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 998, 128)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 127744)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               12774500  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)              

# New Section